In [15]:
from sqlalchemy import create_engine, desc
from idetect.model import db_url, Base, Document, Analysis, Session, Status
import requests

# connect to the DB specified in the docker.env file
engine = create_engine(db_url())
Session.configure(bind=engine)

# create the DB schema, if it doesn't already exist
Base.metadata.create_all(engine)

In [16]:
session = Session()

In [12]:
from os.path import isfile

path = '/home/idetect/data/input_urls.csv'
url = 'https://www.dropbox.com/s/c2vzdzrljlrn3y0/input_urls.csv?dl=1'

if not isfile(path):
    with open(path, 'wb') as f:
        r = requests.get(url, stream=True)
        for chunk in r.iter_content(chunk_size=1024):
            if chunk:  # filter out keep-alive new chunks
                f.write(chunk)

In [13]:
os.path.getsize(path)

78873266

In [22]:
import csv
from datetime import datetime

start = datetime.now()
limit = 1000
i = 0
with open('/home/idetect/data/input_urls.csv') as f:
    c = csv.reader(f)
    while i <= limit
        for l in c:
            url_id, date, url = l
            if not url.startswith('http'):
                continue
            try:
                document1 = Document(url=url, name='test', type='WEB')
                analysis1 = Analysis(document=document1, status=Status.NEW)
                session.add(document1)
                session.commit()
            except:
                print("error on row {}: {}".format(i, l))
                raise
            i += 1
            if i % 1000 == 0:
                print("{} {}".format(i, url))
done_inserting = datetime.now()

1000 http://thewire.in/85372/rebels-seek-ceasefire-syrian-army-closer-retaking-aleppo/
2000 http://news.webindia123.com/news/articles/India/20161103/2982876.html
3000 http://economictimes.indiatimes.com/news/economy/policy/ngt-asks-states-to-submit-worst-hit-cities-pollution-data-in-three-weeks/articleshow/52529458.cms
4000 http://www.fresnobee.com/news/business/article63094577.html
5000 http://poststar.com/news/world/the-latest-germany-seeks-muslim-help-for-migrants/article_114fd340-8d1e-55b7-a26f-c771d6fb663b.html
6000 http://www.news.com.au/national/queensland/state-of-origin-great-johnathan-thurston-sends-message-of-support-to-students-aurukun/news-story/bf843a15462bee0eb0685dd4ef36cb23
7000 http://infotel.ca/newsitem/alberta-ontario-economy/cp231793775
8000 http://www.am1460wixn.com/xnational-news/2016/08/19/maryland-apartment-fire-three-dead-site-of-explosion-identified-1
9000 http://www.news24.com.ng/Africa/News/s-sudan-fighting-sends-refugee-numbers-soaring-past-one-million-mar

KeyboardInterrupt: 

In [ ]:
delta = done_inserting - start

In [ ]:
delta.seconds / 60 / 60